To run by *RunnerPhonon.ipynb*. Does the same as the *Animation.ipynb*, but at the end saves the animation to the current folder.

In [29]:
from matplotlib import pyplot as plt

In [45]:
%store -r

In [46]:
[N,gamma,bc,M1,M2,imp_enabled,Nimp,Mimp,ModeNr,MyInteractionResult]=VariablesForSaveAnim

In [32]:
import matplotlib
from matplotlib import animation, rc
from IPython.display import HTML
import numpy as np
rc('animation', html='jshtml')
#matplotlib.use("Agg")


Initializing values

In [34]:
# #mode=ModeNrInter.value
if (MyInteractionResult is None): #Before all widgets are moved MyInteraction.result==None. Or when N==2 and bc==1
    if (N == 2): #The problematic case of 
        Vanim=V[:,0]
        omegaAnim=omega[0]
    else:
        Vanim=V[:,1]
        omegaAnim=omega[1]
else:
    Vanim=MyInteractionResult[2]
    omegaAnim=MyInteractionResult[1]


omegaAnim=5 if np.abs(omegaAnim)<1e-06 else omegaAnim #Otherwise it will mant to make an infinite animation

Dealing with the case of fixed boundary conditions and N=2. This solves the problem

In [35]:
try:
    len(Vanim)
except:
    Vanim=[[0,0],[0,0]]
    omegaAnim=10

In [36]:
Ampl=0.1*((M1+M2)/gamma)**(1/4) # don't know the reason for exactly ^(1/4)
Vplot=np.array(np.transpose(Vanim)*Ampl)[0]  #transpose and [0] is for some stupid reasons with arrays

oddAtoms = np.arange(1, N+1, 2)
evenAtoms = np.arange(2, N+1, 2)
allAtoms = np.arange(1, N+1)
Vodd=Vplot[::2]
Veven=Vplot[1::2]

yodd = [0]*len(Vodd)
yeven = [0]*len(Veven)

In [37]:
%%capture
#%matplotlib tk
fig2, ax3 = plt.subplots()
fig2.set_size_inches(10.0, 1.4)

ax3.set_xlim(( 0.5, N+0.5))
ax3.set_ylim((-0.05, 0.05))
#fig2.title='MyTitle'


line,=ax3.plot([], [], 'bo', markersize=10)

if M1 > M2:
        mark1 = 11
        mark2 = 6
elif M1 == M2:
        mark1 = mark2 = 6
else:
        mark1 = 6
        mark2 = 11

#Marker colors for masses
if M1==M2:
    col1, col2="blue", "blue"
else:
    col1, col2="blue", "green"
        
        
#plotlays=[2]
plotcols, markers = [col1,col2,"white","red"], [mark1,mark2,11,(np.log(Mimp/M1+1.7)*7)] #["blue","green"],[mark1,mark2] 
lines = []
for index in range(4):
    lobj = ax3.plot([],[],'o',markersize=markers[index],color=plotcols[index])[0]
    lines.append(lobj)
def init():
    for line in lines:
        line.set_data([],[])
    return lines

ax3.set(xlabel="x/a, m", ylabel='y',title='Atomic oscilations, mode nr = {}, bc = {}, $\gamma$ = {}'.format(ModeNr,bc,gamma)) # for N={}, M1={}, M2={},Minp={}.'.format(N,M1,M2,Mimp)

steptick=N//15+1
xTickArray=np.arange(0, N+2, step=steptick)
xTickArray = xTickArray.astype('float64')
xTickArray[0], xTickArray[-1]=0.5, N+0.5
ax3.set_xticks(xTickArray)

fig2.subplots_adjust(bottom=0.3,top=0.8)

Note how big problems the periodic boundary conditions cause(particle can appear from the other side). Otherwise the function bellow bould have been pretty simple. Now you have to look through all the cases and change a bit the  plotting data.

It still does not deal with the case when the impurity is the last particle and it should reapear on the other side. If somebody really creates such a case then congrats, u cool!

In [38]:
def animate(t, *args):
    if bc==0: #periodic boundary conditions (particle can appear from the other side) causes problems
        lines[0].set_data(oddAtoms+np.real(np.exp(-omegaAnim*t*0.005*2*np.pi*1j)*Vodd), yodd)
        lines[1].set_data(evenAtoms+np.real(np.exp(-omegaAnim*t*0.005*2*np.pi*1j)*Veven), yeven)
        
        if np.real(np.exp(-omegaAnim*t*0.005*2*np.pi*1j)*Vodd[0])< -0.5:
            lines[0].set_data(np.append(oddAtoms[1:],N+1)+
                              np.real(np.exp(-omegaAnim*t*0.005*2*np.pi*1j)*np.append(Vodd[1:],Vodd[0])), yodd)
            lines[1].set_data(evenAtoms+np.real(np.exp(-omegaAnim*t*0.005*2*np.pi*1j)*Veven), yeven)
        elif (np.real(np.exp(-omegaAnim*t*0.005*2*np.pi*1j)*Vodd[-1])> 0.5) & (N%2) :
            lines[0].set_data(np.append(0,oddAtoms)+
                              np.real(np.exp(-omegaAnim*t*0.005*2*np.pi*1j)*np.append(Vodd[-1],Vodd)), np.append(0,yodd)) #I don't take out the data point because it is not visible anyway
            lines[1].set_data(evenAtoms+np.real(np.exp(-omegaAnim*t*0.005*2*np.pi*1j)*Veven), yeven)
        elif (np.real(np.exp(-omegaAnim*t*0.005*2*np.pi*1j)*Veven[-1])> 0.5) & (N%2==0) :
            lines[0].set_data(oddAtoms+
                              np.real(np.exp(-omegaAnim*t*0.005*2*np.pi*1j)*Vodd), yodd) #I don't take out the data point because it is not visible anyway
            lines[1].set_data(np.append(0,evenAtoms)+np.real(np.exp(-omegaAnim*t*0.005*2*np.pi*1j)*np.append(Veven[-1],Veven)), np.append(0,yeven))
    else:
        lines[0].set_data(oddAtoms+np.cos(omegaAnim*t*0.005*2*np.pi)*Vodd, yodd)
        lines[1].set_data(evenAtoms+np.cos(omegaAnim*t*0.005*2*np.pi)*Veven, yeven)
        

    if imp_enabled == 1:
        lines[2].set_data(Nimp+np.real(np.exp(-omegaAnim*t*0.005*2*np.pi*1j)*Vplot[Nimp-1]),0)
        lines[3].set_data(Nimp+np.real(np.exp(-omegaAnim*t*0.005*2*np.pi*1j)*Vplot[Nimp-1]),0)
    return (lines)

In [ ]:
anim = animation.FuncAnimation(fig2, animate, init_func=init,
                              frames=int(np.round(200/omegaAnim)), interval=4, blit=True) #frames value taken so that the film ends exatly at the end of the period

In [47]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
Name='PhononOscil_Mode_{}_SpringC_{}_N_{}.mp4'.format(ModeNr,gamma,N)

In [43]:
anim.save(Name, writer=writer)